# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies

INFO:tensorflow:Restoring parameters from D:\github\ben\UCBC 2024\Models\bidding-3233000
INFO:tensorflow:Restoring parameters from D:\github\ben\UCBC 2024\Models\binfo-286000
INFO:tensorflow:Restoring parameters from ..\models/lead_model_b/lead-1000000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/lefty_model/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\models/dummy_model/dummy-920000
INFO:tensorflow:Restoring parameters from ..\models/righty_model/righty-1000000
INFO:tensorflow:Restoring parameters from ..\models/decl_model/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = '6.AKJT82.762.K63'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ['1D', '3S']
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '4H',
  'insta_score': 0.2556,
  'expected_score': 186,
  'adjustment': 13},
 {'call': 'PASS',
  'insta_score': 0.6695,
  'expected_score': 33,
  'adjustment': 33}]

In [4]:
auction = ['1D', '3S', '4H', '4S', '5H', 'PASS']
bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

[{'call': '6H', 'insta_score': 0.132, 'expected_score': 557, 'adjustment': 7},
 {'call': 'PASS',
  'insta_score': 0.8391,
  'expected_score': 409,
  'adjustment': 42}]

In [5]:
# you double, North bids 2S, your partner 3H, PASS, back to you ...

bid = bot_bid.bid(["PAD_START", "PASS", "1H", "X", "4H", "PASS", "PASS", "X"])
bid.bid

'PASS'

In [6]:
# what options were considered?

bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.9995}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [7]:
bid.samples

['x.AKJT8x.xxx.Kxx T98xx.x.T98x.T9x AJxx.Q9xxxx.KJ.x KQx..AQxx.AQJ8xx 0.21341',
 'x.AKJT8x.xxx.Kxx 8xxxx.xx.J9x.xxx Ax.Q9xxx.AT8x.T8 KQJT9..KQx.AQJ9x 0.19239',
 'x.AKJT8x.xxx.Kxx T9xx.9.J8xx.8xxx Kxx.Qxxxxx.A9.JT AQJ8x..KQTx.AQ9x 0.07737',
 'x.AKJT8x.xxx.Kxx 8xxx.x.J8xx.J9xx ATxx.Q9xxx.9x.Ax KQJ9.x.AKQT.QT8x 0.03372',
 'x.AKJT8x.xxx.Kxx T9xxx.x.9xx.Txxx Q8x.Q9xxx.AQ8.Jx AKJx.x.KJTx.AQ98 0.02312']

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [8]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands_auction(['PAD_START', 'PAD_START', 'PAD_START','2N','PASS','3C','PASS','3D'])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

17
x.AKJT8x.xxx.Kxx AQxx.xxx.J8x.Jxx T9xxx.x.T9x.T98x KJ8.Q9x.AKQx.AQx
x.AKJT8x.xxx.Kxx Q9xx.Q9x.K98.xxx T8xxx.x.Txx.JT8x AKJ.xxx.AQJx.AQ9
x.AKJT8x.xxx.Kxx K8xx.xx.Txx.QJT9 QTxxx.9x.8x.xxxx AJ9.Qxx.AKQJ9.A8
x.AKJT8x.xxx.Kxx QJ8x.9x.J8x.QJ98 T9xxx.xx.Q9x.Txx AKx.Qxx.AKTx.Axx
x.AKJT8x.xxx.Kxx KQ9x.9xx.8xxx.Qx JTxxx.xx.T.JT9xx A8x.Qx.AKQJ9.A8x
x.AKJT8x.xxx.Kxx KJxx.9x.8x.QT9xx QT9xx.xx.T9x.J8x A8x.Qxx.AKQJx.Ax
x.AKJT8x.xxx.Kxx Q9xx.xx.J9x.Q8xx JT8xx.9x.Q8.J9xx AKx.Qxx.AKTxx.AT
x.AKJT8x.xxx.Kxx KTxx.9xx.Q98.Txx J98xx.xx.xx.QJ8x AQx.Qx.AKJTx.A9x
x.AKJT8x.xxx.Kxx KJxx.xxx.J98.Txx T98xx.x.Tx.QJ98x AQx.Q9x.AKQxx.Ax
x.AKJT8x.xxx.Kxx J9xx.xx.K8x.QJTx QT8xxx.xxx.9x.9x AK.Q9.AQJTx.A8xx
x.AKJT8x.xxx.Kxx JTxx.xx.K8x.QT9x Q98xxx.9xx.JTx.x AK.Qx.AQ9x.AJ8xx
x.AKJT8x.xxx.Kxx K9xx.9xx.9xx.Q9x QJT8xx.xx.JT.8xx Ax.Qx.AKQ8x.AJTx
x.AKJT8x.xxx.Kxx KT8x.9x.Q98x.JTx QJ9xxx.xxx.T.9xx Ax.Qx.AKJxx.AQ8x
x.AKJT8x.xxx.Kxx QT9x.9xx.Qxx.J9x KJ8xxx.xx.98x.8x Ax.Qx.AKJT.AQTxx
x.AKJT8x.xxx.Kxx Jxxx.9xxx.xx.QJx QT98x.x.Q9x

How do we know on which sample 4H is making and on which sample it is not? It's the topic for another tutorial, but basically we could do either (1) solve the sample double dummpy or (2) estimate the outcome using a neural network

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

In [9]:
hands = random_deal().split()
hands # N, E, S, W

['JT7.AJ62.T9.JT87',
 'AQ9832.T87.876.9',
 'K4.Q43.AQ432.Q65',
 '65.K95.KJ5.AK432']

In [10]:
bidder_bots = [BotBid([False, False], hand, models, sampler, False) for hand in hands]

In [11]:
auction = []  # since North deals, we don't need any 'PAD_START'

turn_i = 0  # whose turn is it to bid

while not bidding.auction_over(auction):
    auction.append(bidder_bots[turn_i].bid(auction).bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    
auction

['PASS', '2S', '3D', '3S', 'PASS', 'PASS', 'PASS']